### 决策树分类的方式
(1) ID3算法以信息增益为准则来进行选择划分属性，选择信息增益最大的；  
(2) C4.5算法先从候选划分属性中找出信息增益高于平均水平的属性，再从中选择增益率最高的；  
(3) CART算法使用“基尼指数”来选择划分属性，选择基尼值最小的属性作为划分属性.  

数据集说明：  
（0）年龄：0代表青年，1代表中年，2代表老年；  
（1）有工作：0代表否，1代表是；  
（2）有自己的房子：0代表否，1代表是；  
（3）信贷情况：0代表一般，1代表好，2代表非常好；  
（4）类别(是否给贷款)：no代表否，yes代表是。

代码参考了：https://github.com/Erikfather/Decision_tree-python

In [6]:
import os, sys
import operator

In [7]:
def readTrainData(fileName):
    """
    年龄段：0代表青年，1代表中年，2代表老年；
    有工作：0代表否，1代表是；
    有自己的房子：0代表否，1代表是；
    信贷情况：0代表一般，1代表好，2代表非常好；
    类别(是否给贷款)：0代表否，1代表是
    """
    f = open(fileName, 'r')
    lines = f.readlines()
    labels = ['age', 'have work', 'have house', 'credit']
    dataSet = []
    for line in lines[:]:
        line = line.strip().split(',')
        dataSet.append(line)
    return dataSet, labels

In [8]:
def readTestData(fileName):
    """
    年龄段：0代表青年，1代表中年，2代表老年；
    有工作：0代表否，1代表是；
    有自己的房子：0代表否，1代表是；
    信贷情况：0代表一般，1代表好，2代表非常好；
    类别(是否给贷款)：0代表否，1代表是
    """
    f = open(fileName, 'r')
    lines = f.readlines()
    dataSet = []
    for line in lines[:]:
        line = line.strip().stripplit(',')
        dataSet.append(line)
    return dataSet

In [9]:
def calEntropy(dataMat):
    '''
    计算熵
    gain(D,A) = H(D) - H(D|A)
    这里这个函数是计算H(D)
    '''
    numTrainData = len(dataMat)
    labelCounts = {}
    for featVec in dataset:
        currentlabel=featVec[-1]
        if currentlabel not in labelCounts.keys():
            labelCounts[currentlabel]=0
        labelCounts[currentlabel]+=1
    Ent = 0
    for key in labelCounts:
        p=float(labelCounts[key])/numEntries
        Ent=Ent-p*log(p,2)#以2为底求对数
    return Ent

In [ ]:
def splitDataByFeature(dataMat, axis, value):
    '''
    根据特征划分数据集用于计算条件上熵
    '''
    retDataSet=[]#创建返回的数据集列表
    for featVec in dataset:#抽取符合划分特征的值
        if featVec[axis]==value:
            reducedFeatVec=featVec[:axis] #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])#将符合条件的特征添加到返回的数据集列表
            retDataSet.append(reducedFeatVec)
    return retDataSet
    
    
def ID3(dataMat):
    '''
    ID3算法:以信息增益为准则选择划分属性
    gain(D,A) = H(D) - H(D|A)
    这里计算gain(D,A)
    '''
    maxInfoGain = 0
    bestFeature = -1
    numFeatures = len(dataMat[0]) - 1
    Ent = calEntropy(dataMat)
    for i in range(numFeatures):
        featList = [example[i] for example in dataMat]
        uniqueVals=set(featList) #将特征列表创建成为set集合，元素不可重复。创建唯一的分类标签列表
        for value in uniqueVals:     #计算每种划分方式的信息熵
            subDataMat = splitDataByFeature(dataMat, i, value)
            p = len(subDataMat) / float(len(dataMat))
            featureEnt = p * calEntropy(subDataMat)
        infoGain = Ent - featureEnt
        print(u"ID3中第%d个特征的信息增益为：%.3f"%(i,infoGain))
        if(infoGain > maxInfoGain):
            maxInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [10]:
readTrainData('./data/dataset.txt')

([['0', '0', '0', '0', '0'],
  ['0', '0', '0', '1', '0'],
  ['0', '1', '0', '1', '1'],
  ['0', '1', '1', '0', '1'],
  ['0', '0', '0', '0', '0'],
  ['1', '0', '0', '0', '0'],
  ['1', '0', '0', '1', '0'],
  ['1', '1', '1', '1', '1'],
  ['1', '0', '1', '2', '1'],
  ['1', '0', '1', '2', '1'],
  ['2', '0', '1', '2', '1'],
  ['2', '0', '1', '1', '1'],
  ['2', '1', '0', '1', '1'],
  ['2', '1', '0', '2', '1'],
  ['2', '0', '0', '0', '0'],
  ['2', '0', '0', '2', '0']],
 ['age', 'have work', 'have house', 'credit'])